In [1]:
from fireworks.ic import ic_two_body as ic_two_body
from fireworks.ic import ic_random_uniform as ic_random_uniform

from fireworks.nbodylib import dynamics as dyn
from fireworks.nbodylib import integrators as intg


import matplotlib.pyplot as plt
import numpy as np
from typing import Optional, Tuple
import numpy.typing as npt
from fireworks.particles import Particles


import numba
from numba import prange, njit

In [39]:
mass1 = 1.0
mass2 = 1.0
rp = 1.0
e = 0.0

particles = ic_two_body(mass1, mass2, rp, e)

pos = particles.pos
vel = particles.vel
N_particles = len(particles)  
N = N_particles  
mass = particles.mass
softening = 1e-3
tstep = 1e-3

#particles = ic_random_uniform(100,[1,1],[1,10],[0,1])

In [28]:
a = 0 
b = 2

In [29]:
dx = pos[a:b, 0,np.newaxis] - pos[:, 0] #broadcasting of (N,) on (N,1) array, obtain distance along x in an (N,N) matrix
dy = pos[a:b, 1,np.newaxis] - pos[:, 1] 
dz = pos[a:b, 2,np.newaxis] - pos[:, 2] 
    
r = np.sqrt(dx**2 + dy**2 + dz**2)
r[r==0]=1

dpos = np.concatenate((dx, dy, dz)).reshape((3,N_particles,N_particles)) 



In [30]:
dpos.shape

(3, 2, 2)

In [35]:

def full_evo(a,b,particles):
   
    # Questo può essere così oppure rendo global direttamente particles...
    
    N_subset = abs(b-a)

    pos = particles.pos
    vel = particles.vel
    N_particles = len(particles)
    mass = particles.mass

    # Select particles from a to b to parallelize computation
    # Need to rewrite the function in order to compute quantities of subset of particles wrt all the others
    dx = pos[a:b, 0,np.newaxis] - pos[:, 0] #broadcasting of (N,) on (N,1) array, obtain distance along x in an (N,N) matrix
    dy = pos[a:b, 1,np.newaxis] - pos[:, 1] 
    dz = pos[a:b, 2,np.newaxis] - pos[:, 2] 
      
    r = np.sqrt(dx**2 + dy**2 + dz**2)
    r[r==0]=1
    # New dpos shape is (3,N_particles,3000) since 
    # 3 is the number of dimensions, 
    # N_subset is the number of particles in the subset and
    # N_particles is the number of total particles
    # dpos is the distance vector between each particle in the subset and all the others

    dpos = np.concatenate((dx, dy, dz)).reshape((3,N_subset,N_particles)) 
    
    acc = - (dpos/r**3 @ mass).T
    jerk= None
    pot = None

    return acc, jerk, pot

In [44]:
particles = ic_random_uniform(3000,[1,1],[1,10],[0,1])

N_particles = len(particles)
mass = particles.mass
pos = particles.pos
vel = particles.vel

full_evo(0,750,particles)


(array([[-0., -0., -0.],
        [-0., -0., -0.],
        [-0., -0., -0.],
        ...,
        [-0., -0., -0.],
        [-0., -0., -0.],
        [-0., -0., -0.]]),
 None,
 None)

In [40]:
def acceleration_direct_vectorized(N_particles, pos, mass):
   
    dx = pos[:, 0].reshape(N_particles, 1) - pos[:, 0] #broadcasting of (N,) on (N,1) array, obtain distance along x in an (N,N) matrix
    dy = pos[:, 1].reshape(N_particles, 1) - pos[:, 1] 
    dz = pos[:, 2].reshape(N_particles, 1) - pos[:, 2] 
      
    r = np.sqrt(dx**2 + dy**2 + dz**2)
    r[r==0]=1
    
    dpos = np.concatenate((dx, dy, dz)).reshape((3,N_particles,N_particles)) 


    acc = - (dpos/r**3 @ mass).T
    jerk= None
    pot = None

    return acc, jerk, pot

In [47]:

acceleration_direct_vectorized(N_particles, pos, mass)

(array([[-0., -0., -0.],
        [-0., -0., -0.],
        [-0., -0., -0.],
        ...,
        [-0., -0., -0.],
        [-0., -0., -0.],
        [-0., -0., -0.]]),
 None,
 None)

In [ ]:
@njit(parallel=True,fastmath=True)
def fast_acceleration_direct_vectorized(N_particles, pos, mass):
   
    dx = pos[:, 0].copy().reshape(N_particles, 1) - pos[:, 0] #broadcasting of (N,) on (N,1) array, obtain distance along x in an (N,N) matrix
    dy = pos[:, 1].copy().reshape(N_particles, 1) - pos[:, 1] 
    dz = pos[:, 2].copy().reshape(N_particles, 1) - pos[:, 2] 
      
    r = np.sqrt(dx**2 + dy**2 + dz**2)
    #r[r==0]=1 not supported on numba
    r += np.eye(r.shape[0])
    
    dpos = np.concatenate((dx, dy, dz)).copy().reshape((3,N_particles,N_particles)) 

    acc = np.zeros((3,N_particles),dtype=np.float64)
    np.sum((dpos/r**3 * mass),axis=2) #not supported on numba
    

    acc = -acc.T
    jerk= None
    pot = None

    return acc, jerk, pot

In [ ]:
#%%timeit
fast_acceleration_direct_vectorized(N, pos, mass)